<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Postprocessing Tutorial
FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).



This notebook will show you the basic postprocessing features that FAST-OAD offers.

**Note: The 01_tutorial notebook has to be ran to generate the results files used in this postprocessing notebook**

*Tip: hover your mouse pointer over any plot to inspect it or get menu to manipulate it*

In [2]:
import os.path as pth
from fastoad.io import VariableIO
from fastoad.gui.analysis_and_plots import wing_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots import (
    mass_breakdown_sun_plot,
    mass_breakdown_bar_plot,
    aircraft_geometry_plot,
)

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.propulsion.hybrid_propulsion.basicHE_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_hybrid_powertrain.py
Failed to import module fastga.models.hybrid_powertrain.test_hybrid_powertrain.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_

To illustrate the set of postprocessing plots we will use the design obtained in the `01_tutorial` notebook:
- Fuel cell aircraft 150nm Multidisciplinary Design Analysis (MDA)

In [3]:
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

Fuel_cell_150nm_MDA_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_fuel_cell_150NM_mda.xml"
)
Beechcraft_1000nm_MDA_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_hq_work.xml"
)
# Beechcraft_800nm_MDO_OUTPUT_FILE = pth.join(
#     WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mdo.xml"
# )

## 1. Introduction

The general philosophy of the postprocessing plots is to use the result data file as the source information. Hence, offline of the computational process. Furthermore, the idea is to compare design results by enabling the superposition of different plots for the postprocessing plots where comparison is relevant.

## 2. Geometry plots
For instance here, we use the `fig` variable generated by the code line 1 and plot the result of line 2 on the existing figure. There is no number of maximum superpositions.

In [3]:
fig = wing_geometry_plot(Fuel_cell_150nm_MDA_OUTPUT_FILE, name="Fuel_cell 150nm MDA")
# fig = wing_geometry_plot(Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig)
# fig = wing_geometry_plot(Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig)
fig.show()

In [4]:
fig = aircraft_geometry_plot(Fuel_cell_150nm_MDA_OUTPUT_FILE, name="Fuel cell 150 nm MDA")
fig = aircraft_geometry_plot(
    Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig
)
# fig = aircraft_geometry_plot(
#     Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig
# )
fig.show()

## 3. Mass breakdown plots

This first mass breakdown plot enables to visualize the Maximum TakeOff Weight (MTOW) and Overall Weight Empty (OWE) for a single design.

In [5]:
fig = mass_breakdown_sun_plot(Fuel_cell_150nm_MDA_OUTPUT_FILE)
fig.show()

This second mass breakdown plot provides less detail but enables to compare designs.

## 4. Mission plots

In [6]:
fig = mass_breakdown_bar_plot(Fuel_cell_150nm_MDA_OUTPUT_FILE, name="Beechcraft 800 nm MDA")
# fig = mass_breakdown_bar_plot(
#     Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig
# )
# fig = mass_breakdown_bar_plot(
#     Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig
# )
fig.show()

An other post-processing tool is available: the mission viewer. You simply add the mission files that interest you:

In [ ]:
from fastoad.gui.mission_viewer import MissionViewer

Beechcraft_800nm_MDA_MISSION_FILE = pth.join(WORK_FOLDER_PATH, "workdir/mda_mission_results.csv")

mission = MissionViewer()
mission.add_mission(Beechcraft_800nm_MDA_MISSION_FILE, name="Beechracft 800 nm MDA")

mission.missions["Beechracft 800 nm MDA"]

Or you can plot the missions:

In [ ]:
mission.display()

## 5. Payload-Range Diagram

FAST-OAD-GA also allows the user to display some interresting plot from an engineering point of view. One such example is the payload-range diagram. Such diagram, though useful for assessing the performances of the aircraft, are not mandatory for the MDA to converge, so they are not included in the base configuration file. This also means that before plotting the diagram we will have to run a special simulation on the file obtained at the output of the MDA before being able to display the actual plot

As we did in the first tutorial, let's set up a run with only the module that computes the payload range diagram (it can take up to 10 min).

In [ ]:
import shutil

from fastoad import api as api_cs25

CONFIGURATION_FILE_DATA = pth.join(DATA_FOLDER_PATH, "payload_range.yml")
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "payload_range.yml")

SOURCE_FILE_PAYLOAD_RANGE = pth.join(WORK_FOLDER_PATH, "payload_range.xml")

# First copy the configuration file inside the workfolder, and create a duplicate of the MDA outputs

shutil.copy(CONFIGURATION_FILE_DATA, CONFIGURATION_FILE)
shutil.copy(Beechcraft_800nm_MDA_OUTPUT_FILE, SOURCE_FILE_PAYLOAD_RANGE)

# Then generate the inputs

api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE_PAYLOAD_RANGE, overwrite=True)

eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

We will now save the outputs and use the proper post-processing function

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import payload_range

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "payload_range_outputs.xml")

fig = payload_range(OUTPUT_FILE, name="Beechcraft 800 nm MDA")
fig.show()

## 6. Aircraft polars

Another diagram that we can plot is the aircraft polars (be they equilibrated or not equilibrated. As for the payload-range we first need to launch the analysis before plotting them. This time, it should only take a couple seconds.

In [ ]:
import shutil

from fastoad import api as api_cs25

CONFIGURATION_FILE_DATA = pth.join(DATA_FOLDER_PATH, "aircraft_polar.yml")
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "aircraft_polar.yml")

SOURCE_FILE_POLAR = pth.join(WORK_FOLDER_PATH, "polar_inputs.xml")

# First copy the configuration file inside the workfolder, and create a duplicate of the MDA outputs

shutil.copy(CONFIGURATION_FILE_DATA, CONFIGURATION_FILE)
shutil.copy(Beechcraft_800nm_MDA_OUTPUT_FILE, SOURCE_FILE_POLAR)

# Then generate the inputs

api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE_POLAR, overwrite=True)

eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

As before we will now save the outputs and use the proper post-processing function as before. Let's start with the non equilibrated one.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_polar

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "polar_outputs.xml")

fig = aircraft_polar(OUTPUT_FILE, name="Beechcraft 800 nm MDA")
fig.show()

And let's now do the equilibrated one.

In [ ]:
fig = aircraft_polar(OUTPUT_FILE, name="Beechcraft 800 nm MDA", equilibrated=True)
fig.show()